### 1. Скачайте текст войны и мир (ссылка на github и гугл диск). Очистите его, оставив только русские буквы и символы пробела. Постройте матрицу переходных вероятностей. Посчитайте вероятность встретить строку из 4-х символов, которая начинается п, а кончается на р; Используя матрицу переходных вероятностей сгенерируйте слово, начинающееся с буквы “Б” и имеющее длину 8-мь символов
### 2. Выберите ряд и оцените его через HMM. Попробуйте “построить” (через среднее каждой компоненты) наивный прогноз ряда

In [1]:
import os
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
sns.set()

In [2]:
just_text=''
for filename in os.listdir(os.getcwd()):
    #print(filename)
    if '.txt' in filename:
        with open(filename, 'r') as f:
           #print(f.read().lower()[:509])
            just_text+=f.read().lower()
        #just_text=
        #print(filename)

In [3]:
#только русские буквы и символы пробела

In [4]:
import re
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [5]:
text=words_only(just_text)

In [6]:
text[:1000]

'лев толстой война и мир том том часть первая часть орая часть третья том лев николаевич толстой война и мир том часть первая е поместья мой верный раб ну что князь генуа и лукка стали не больше как поместьями фамилии бонапарте нет я вас предупреждаю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист я вас больше не знаю вы уж не друг мой вы уж не мой верный раб как вы говорите ну здравствуйте здравствуйте я вижу что я вас пугаю садитесь и рассказывайте так говорила в июле года известная анна павловна шерер фрейлина и приближенная императрицы марии феодоровны встречая важного и чиновного князя василия первого приехавшего на ее вечер анна павловна кашляла несколько дней у нее был грипп как она говорила грипп был тогда новое слово употреблявшееся только редкими в записочках разосланных утром с красным лакеем было написано без различия во всех или если вас граф или князь нет в виду ничего лучшего 

In [7]:
#Лемматизация

In [8]:
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [ ]:
text_lemma=lemmatize(text)

In [ ]:
text_lemma[:1000]

### Постройте матрицу переходных вероятностей

In [ ]:
RUSSIAN = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'

POS = {l: i for i, l in enumerate(RUSSIAN)}

probabilities = np.ones((len(RUSSIAN), len(RUSSIAN)))

for cp, cn in zip(text[:-1], text[1:]):
    if (cp!=' ') and (cn!=' '):
        probabilities[POS[cp], POS[cn]] += 1

probabilities = (probabilities / probabilities.sum(axis=1)[:, None])
    

df = pd.DataFrame(probabilities, index=(c for c in RUSSIAN), columns=(c for c in RUSSIAN))

plt.figure(figsize=(16,10))
sns.heatmap(df, xticklabels=True, yticklabels=True)
plt.tight_layout()
plt.show()

### Посчитайте вероятность встретить строку из 4-х символов, которая начинается п, а кончается на р

In [ ]:
start = POS['п']
end = POS['р']
prior = 1. / len(RUSSIAN)
proba_sum = 0.
for c1 in RUSSIAN:
    for c2 in RUSSIAN:
        proba = prior * probabilities[start, POS[c1]] * probabilities[POS[c1], POS[c2]] * probabilities[POS[c2], end]
        proba_sum += proba

In [ ]:
proba_sum

### Используя матрицу переходных вероятностей сгенерируйте слово, начинающееся с буквы “Б” и имеющее длину 8-мь символов

In [ ]:
s = []
start = 'б'
s.append(start)
length = 8
for i in range(length):
    index = POS[s[i]]
    next_char = np.random.choice(list(RUSSIAN), p=probabilities[index])
    s.append(next_char)

print (''.join(s))

### Выберите ряд и оцените его через HMM. Попробуйте “построить” (через среднее каждой компоненты) наивный прогноз ряда

In [ ]:
from hmmlearn.hmm import GaussianHMM
import warnings
warnings.filterwarnings('ignore')

In [ ]:
series = pd.read_csv('monthly-sales-of-company-x-jan-6.csv',sep=',').iloc[:,1]

In [ ]:
values = series.values.reshape(-1,1)

In [ ]:
X_train = values[:150]
X_test = values[150:]

In [ ]:
for n in range(1, 20):
    model = GaussianHMM(n_components=n,
                        covariance_type="diag",
                        n_iter=1000)
    model.fit(X_train)
    print (n, model.score(X_train))

In [ ]:
model = GaussianHMM(n_components=29,
                        covariance_type="diag",
                        n_iter=1000)
model.fit(X_train)
labels = model.predict(values)
means = np.zeros_like(values)
for i in range(model.n_components):
    means[labels == i] = model.means_[i]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(values,label='values')
plt.plot(means, linewidth=4, alpha=0.45,linestyle='-.', color='red',label='means')
plt.legend()
plt.show()